## Setup
1- Clone nidaqmx simulator from https://github.com/TheWiselyBearded/nidaqmx-python

2- Navigate to cloned directory and run `pip install -e .`

3- Run notebook w/ simulator attached.

In [ ]:
import time
import datetime
from collections import OrderedDict
import threading
import random
import collections
import os
import numpy as np
import nidaqmx
import pickle
import sys

import quantities as pq
from nidaqmx.stream_writers import (
    DigitalSingleChannelWriter, DigitalMultiChannelWriter)
from nidaqmx.utils import flatten_channel_string
from nidaqmx.constants import (
    LineGrouping, AcquisitionType, DigitalWidthUnits, Edge,
    HandshakeStartCondition, Level, MIOAIConvertTimebaseSource,
    OverflowBehavior, TaskMode, Polarity, RegenerationMode,
    SampleInputDataWhen, SampleTimingType, UnderflowBehavior)
from nidaqmx.error_codes import DAQmxErrors, DAQmxWarnings
from nidaqmx.errors import (
    check_for_error, is_string_buffer_too_small, DaqError, DaqResourceWarning)

from scipy.optimize import fsolve, least_squares
np.set_printoptions(precision=4)

from olfactometer.logical_olfactometer import LogicalOlfactometer
from olfactometer.equipment import AirSupply, YConnector, Mask, Olfactometer
from olfactometer.my_equipment import MyManifold, MyValve, MyJar, MyLowMFC, \
                                      MyMediumMFC, MyHighMFC, PTFETube, StiffTube, \
                                      STUB_LENGTH
from olfactometer.odorants import Solution, Compound, ChemicalOrder, \
                                  Vendor, Molecule
# from graph import make_graph, draw_graph
from pprint import pprint
from olfactometer.smell_engine_communicator import SmellEngineCommunicator
from olfactometer.data_container import DataContainer
from olfactometer.ui import UI
from olfactometer.PID_tester import PID_Tester
from IPython.display import display

### Specify molecules + dilutions

In [ ]:
#molecules = OrderedDict([(439250, 'l-limonene'), (439570, 'l-carvone'), (440917, 'd-limonene')])
#molecule_dilutions = [10, 100, 10]
#molecules = OrderedDict([(7410, 'Acetophenone'), (7439, 'carvone'), (440917, 'd-limonene')])
molecules = OrderedDict([(702, 'Ethanol')])
molecule_dilutions = [10]
#molecule_dilutions = [10, 1, 10]
# Initialize UI
ui = UI(molecules, print_PID_average=False)

### Initialize system w/ debug_mode(sim)

In [ ]:
smell_engine = SmellEngine(1000, len(molecules), DataContainer(), debug_mode=True, write_flag=False, PID_mode = False, oms=molecules)  # WRITE MODE ENABLED
smell_engine.set_odorant_molecule_ids(list(molecules.keys()))
smell_engine.set_odorant_molecule_dilutions(molecule_dilutions)
smell_engine.initialize_smell_engine_system()
smell_engine.olf.loaded_molecules

### Init concentration sliders

In [ ]:
print(smell_engine.olf.jars.items())
print(smell_engine.olf.optimized_values)
#for j, _ in self.jars.items():
    

In [ ]:
ui.odorConcentrationUI()

In [ ]:
# SPECIFYING CONCENTRATIONS WITHOUT THREADED SLIDERS
concentration_mixtures = ui.odorConcentrationValues() # Read in user-specified concentrations
# Assign target concentrations which runs optimizer
smell_engine.set_desired_concentrations(concentration_mixtures)  

In [ ]:
smell_engine.set_desired_concentrations([1e-9, 1e-9, 1e-9]) 
smell_engine.set_desired_concentrations([1e-7, 1e-9, 1e-9]) 
smell_engine.set_desired_concentrations([1e-6, 1e-9, 1e-9]) 
smell_engine.set_desired_concentrations([1e-5, 1e-9, 1e-9]) 
smell_engine.set_desired_concentrations([1e-3, 1e-9, 1e-9]) 
smell_engine.set_desired_concentrations([1e-9, 1e-7, 1e-7]) 
smell_engine.set_desired_concentrations([1e-7, 1e-7, 1e-7]) 
smell_engine.set_desired_concentrations([1e-6, 1e-7, 1e-7]) 
smell_engine.set_desired_concentrations([1e-5, 1e-7, 1e-7]) 
smell_engine.set_desired_concentrations([1e-3, 1e-7, 1e-7]) 
smell_engine.set_desired_concentrations([1e-9, 1e-6, 1e-6]) 
smell_engine.set_desired_concentrations([1e-7, 1e-6, 1e-6]) 
smell_engine.set_desired_concentrations([1e-6, 1e-6, 1e-6]) 
smell_engine.set_desired_concentrations([1e-5, 1e-6, 1e-6]) 
smell_engine.set_desired_concentrations([1e-3, 1e-6, 1e-6]) 

In [ ]:
smell_engine.olf.write_out_data()

In [ ]:
x= [-1.96417781e+04,  7.24018364e-09,  7.23739848e-09,  7.24148372e-09,
        7.24148372e-09,  7.24358596e-09,  7.23564058e-09,  7.24358596e-09,
        7.24148372e-09,  7.24148372e-09, -4.61896670e+00,  7.14479478e-09,
        7.14479472e-09,  7.14479481e-09,  7.14479480e-09,  7.14479484e-09,
        7.14479467e-09,  7.14479485e-09,  7.14479480e-09,  7.14479480e-09,
       -9.10437587e-11, -8.25071524e-09]

smell_engine.olf.opt_report_nl_solver(x)

### To plot contours
Copy/paste `variables` and `x` which is outputted from the optimizer call in the cell above (setting desired concentration) 

In [ ]:
%matplotlib widget
# for 1 picomols
# Variables represents output of NLLSQ
variables = [0.07444331758826919, 0.10274727290256475, 0.10274727290256475, 0.10274727290256475, 0.10274727290256475, 0.10274727290256475, 0.10274727290256475, 0.10274727290256475, 0.10274727290256475, 0.10274727290256475, 0.09973304338765325, 0.10002964932080552, 0.10002964932080552, 0.10002964932080552, 0.10002964932080552, 0.10002964932080552, 0.10002964932080552, 0.10002964932080552, 0.10002964932080552, 0.10002964932080552, 0.07153918220374905, 0.0997032425152371]
# x is optimal flux as outputted from LS
x = [5.432296684615908, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
# Passing variables followed by index of variables to generate contour plot
# So, 0 => Valve 1 A and 10 => Valve 1 B
smell_engine.olf.mutlidim_plotting.graph_contour_points(variables,0,21,x)
print(smell_engine.olf.mutlidim_plotting.constants_text(variables,0,21,x))